This is just a test

In [1]:
from keras.models import Sequential, Model
from keras.layers import LSTM, GRU, SimpleRNN, RepeatVector, TimeDistributed, Dense, Input, Lambda
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
ALPHABET = '0123456789- '
DIGITS = 3
TRAINING_SERIES_LEN = 3

In [3]:
HIDDEN_SIZE = 256
EPOCHS = 50
ALPHABET_LENGTH = len(ALPHABET)
MAXLEN = (DIGITS + 1) * TRAINING_SERIES_LEN + (TRAINING_SERIES_LEN - 1) 

In [4]:
class OneHot(object):
    def __init__(self, characters):
        self.chars = sorted(set(characters))
        self.char_to_index = dict((char, i) for i, char in enumerate(self.chars))
        self.index_to_char = dict((i, char) for i, char in enumerate(self.chars))

    def encode(self, string, length):
        enc = np.zeros((length, len(self.chars)))
        for i, char in enumerate(string):
            enc[i, self.char_to_index[char]] = 1
        for i in range(len(string), length):
            enc[i, self.char_to_index[' ']] = 1
        return enc
    
    def decode(self, mat):            
        return "".join([self.decode_line(mat[x]) for x in range(mat.shape[0])])
    
    def decode_line(self, vec):
        return self.index_to_char[np.argmax(vec)]

In [5]:
def create_data(size, train_series_len, max_exp=3, min_factor=-9, max_factor=9, digits=DIGITS):
    X = []
    y = []
    hist = []
    max_value = 10 ** digits - 1
    min_value = -max_value
    value_store = set()
    while len(X) < size:
        max_exp = np.random.randint(low=1, high=max_exp + 1)
        factors = np.random.randint(low=min_factor, high=max_factor + 1, size=max_exp + 1)
        
        fun = lambda _i: sum(f * (_i ** x) for x, f in enumerate(factors))
        
        series = tuple(fun(i) for i in range(train_series_len + 1))
        
        if not series in value_store and all(min_value <= x <= max_value for x in series):
            X.append(" ".join([str(x) for x in series[:-1]]))
            y.append(str(series[-1]))
        
    return X, y

In [6]:
X, y = create_data(100000, TRAINING_SERIES_LEN)

encoder = OneHot(ALPHABET)
for i in range(len(X)):
    X[i] = encoder.encode(X[i], MAXLEN)
    y[i] = encoder.encode(y[i], DIGITS + 1)
    
print(f"Created {len(X)} data points")

Created 100000 data points


In [7]:
X = np.array(X)
y = np.array(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val  :", X_val.shape)
print("y_val  :", y_val.shape)

X_train: (80000, 14, 12)
y_train: (80000, 4, 12)
X_val  : (20000, 14, 12)
y_val  : (20000, 4, 12)


In [8]:
lstn_input_shape = (MAXLEN, ALPHABET_LENGTH)
print(lstn_input_shape)

def create_model():

    _model = Sequential()
    _model.add(LSTM(HIDDEN_SIZE, input_shape=lstn_input_shape)) # Encoder
    _model.add(RepeatVector(DIGITS + 1))  # Stellt dem RNN im nächsten Schritt die Ausgabe des vorherigen bereit
    _model.add(LSTM(HIDDEN_SIZE, return_sequences=True)) # Decoder
    _model.add(TimeDistributed(Dense(ALPHABET_LENGTH, activation='softmax'))) # Wendet eine `Dense` Schicht auf jede Ausgabe des Decoders an
                                    # und ermittelt mittels 'softmax'-Funktion die Auswahlwahrscheinlichkeiten der Zeichen
    _model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return _model

(14, 12)


In [9]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [10]:
def train(model, X_train, y_train, X_val, y_val, encoder, epochs):
    loss, acc, val_loss, val_acc = [], [], [], []
    for iteration in range(1, epochs + 1):
        print('-' * 50)
        print('Iteration', iteration)
        hist = model.fit(X_train, y_train,
                  batch_size=1000,
                  epochs=1,
                  validation_data=(X_val, y_val))
        loss.append(hist.history['loss'])
        acc.append(hist.history['accuracy'])
        val_loss.append(hist.history['val_loss'])
        val_acc.append(hist.history['val_accuracy'])
        if iteration % 5 == 0:
            for i in range(10):
                ind = np.random.randint(0, X_val.shape[0])
                rowx, rowy = X_val[ind], y_val[ind]
                strx = ''
                for seqx in rowx:
                    strx += encoder.decode_max(seqx)
                stry = ''
                for seqy in rowy:
                    stry += encoder.decode_max(seqy)
                preds = model.predict_classes([[rowx]], verbose=0)
                strpred = ''
                for p in preds[0]:
                    strpred += encoder.index_to_char[p]
                if stry.strip() == strpred.strip():
                    print('{}{} = {} ☑ {}{}'.format(colors.ok, strx, stry, strpred, colors.close))
                else:
                    print('{}{} = {} ☒ {}{}'.format(colors.fail, strx, stry, strpred, colors.close))
    return loss, acc, val_loss, val_acc

In [11]:
model = create_model()

print("Training model:")
print(model.summary())

loss, acc, val_loss, val_acc = train(model, X_train, y_train, X_val, y_val, encoder, EPOCHS)

Training model:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               275456    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 256)            525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 12)             3084      
Total params: 803,852
Trainable params: 803,852
Non-trainable params: 0
_________________________________________________________________
None
--------------------------------------------------
Iteration 1
80/80 [==============================] - 90s 1s/step - loss: 1.7197 - accuracy: 0.4515 - val_loss: 1.1420 - val_accuracy: 0.6397
-----------------------------------

KeyboardInterrupt: 